In [40]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob


In [37]:
!pip install tensorflow
!pip install keras

In [38]:
!pip install scipy

In [39]:
!pip install glob2


In [41]:
IMAGESHAPE = [224, 224, 3]
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test'


In [42]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)


In [12]:
!ls -lh /root/.keras/models


total 57M
-rw-r--r-- 1 root root 57M May 29 15:58 vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [43]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False


In [44]:
classes = glob('chest_xray/train/*')


In [45]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [58]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/chest_xray/train'  # Ensure this is the correct path


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Path to your dataset
data_path = '/content/drive/MyDrive/chest_xray/train'  # Replace with your actual dataset path
data_p ='/content/drive/MyDrive/chest_xray/val'
# Parameters
batch_size = 32
img_height = 224
img_width = 224
validation_split = 0.2
seed = 123
image_size = (img_height, img_width)

# Load the training dataset
train_dataset = image_dataset_from_directory(
    data_path,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size
)

# Load the validation dataset
validation_dataset = image_dataset_from_directory(
    data_p,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size
)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# Normalization layer
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

# Apply the transformations to the training dataset
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))

# Apply normalization to the validation dataset
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

# Optimize dataset loading
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Define the model
num_classes = 2  # Replace 2 with the actual number of classes in your dataset

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_dataset)
print(f"Validation accuracy: {accuracy * 100:.2f}%")


Found 5211 files belonging to 2 classes.
Using 4169 files for training.
Found 16 files belonging to 2 classes.
Using 3 files for validation.
Epoch 1/10
131/131 [==============================] - 848s 4s/step - loss: 0.5577 - accuracy: 0.7292 - val_loss: 0.3245 - val_accuracy: 0.6667
Epoch 2/10
131/131 [==============================] - 486s 4s/step - loss: 0.4155 - accuracy: 0.7836 - val_loss: 0.5116 - val_accuracy: 0.6667
Epoch 3/10
131/131 [==============================] - 493s 4s/step - loss: 0.3244 - accuracy: 0.8429 - val_loss: 0.2955 - val_accuracy: 0.6667
Epoch 4/10
131/131 [==============================] - 487s 4s/step - loss: 0.2583 - accuracy: 0.8952 - val_loss: 0.1097 - val_accuracy: 1.0000
Epoch 5/10
131/131 [==============================] - 487s 4s/step - loss: 0.2331 - accuracy: 0.9084 - val_loss: 0.0711 - val_accuracy: 1.0000
Epoch 6/10
131/131 [==============================] - 482s 4s/step - loss: 0.1962 - accuracy: 0.9247 - val_loss: 0.0677 - val_accuracy: 1.0000
E

In [ ]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


In [64]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [65]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [66]:
training_set = train_datagen.flow_from_directory('chest_xray/train',
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')



FileNotFoundError: [Errno 2] No such file or directory: 'chest_xray/train'

In [ ]:
test_set = testing_datagen.flow_from_directory('chest_xray/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')


In [ ]:
fitted_model = final_model.fit_generator(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)


In [ ]:
final_model.save('our_model.h5')


In [ ]:
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('our_model.h5') #Loading our model
img=image.load_img('D:/Semester - 6/PneumoniaGFG/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')


In [ ]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16 #Import all the necessary modules
import matplotlib.pyplot as plot
from glob import glob

IMAGESHAPE = [224, 224, 3] #Provide image size as 224 x 224 this is a fixed-size for VGG16 architecture
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)
#3 signifies that we are working with RGB type of images.
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test' #Give our training and testing path

for each_layer in vgg_model.layers:
	each_layer.trainable = False #Set the trainable as False, So that all the layers would not be trained.
classes = glob('chest_xray/train/*') #Finding how many classes present in our train dataset.
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)
final_model = Model(inputs=vgg_model.input, outputs=prediction) #Combine the VGG output and prediction , this all together will create a model.
final_model.summary() #Displaying the summary
final_model.compile( #Compiling our model using adam optimizer and optimization metric as accuracy.
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, #importing our dataset to keras using ImageDataGenerator in keras.
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)
training_set = train_datagen.flow_from_directory('chest_xray/train', #inserting the images.
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')
test_set = testing_datagen.flow_from_directory('chest_xray/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')
fitted_model = final_model.fit_generator( #Fitting the model.
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)
plot.plot(fitted_model.history['loss'], label='training loss') #Plotting the accuracies
plot.plot(fitted_model.history['val_loss'], label='validation loss')
plot.legend()
plot.show()
plot.savefig('LossVal_loss')
plot.plot(fitted_model.history['acc'], label='training accuracy')
plot.plot(fitted_model.history['val_acc'], label='validation accuracy')
plot.legend()
plot.show()
plot.savefig('AccVal_acc')
final_model.save('our_model.h5') #Saving the model file.


In [ ]:
#Test Model
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('our_model.h5') #Loading our model
img=image.load_img('D:/Semester - 6/PneumoniaGFG/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')
